In [2]:
import cv2
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def three_step_search_optimized(current_block, previous_gray, row, col, block_size, search_range):
    """
    Three Step Search (TSS) ottimizzato con Numba per trovare il miglior match.
    """
    h, w = previous_gray.shape
    best_row = row
    best_col = col
    best_sad = float('inf')
    p = search_range

    for step in range(3):
        for dy in range(-p, p + 1, p):
            for dx in range(-p, p + 1, p):
                candidate_row = row + dy
                candidate_col = col + dx

                if (candidate_row < 0 or candidate_col < 0 or
                    candidate_row + block_size > h or candidate_col + block_size > w):
                    continue

                # Calcolo della SAD tra il blocco corrente e il blocco candidato
                sad = 0.0
                for i in range(block_size):
                    for j in range(block_size):
                        sad += abs(current_block[i, j] - previous_gray[candidate_row + i, candidate_col + j])

                if sad < best_sad:
                    best_sad = sad
                    best_row = candidate_row
                    best_col = candidate_col

        p = max(1, p // 2)

    return best_sad, best_row, best_col

@njit(parallel=True)
def compute_motion_mask(current_gray, previous_gray, block_size, search_range, diff_threshold):
    """
    Calcola la maschera di movimento usando il Three Step Search (TSS).
    """
    h, w = current_gray.shape
    mask = np.zeros((h, w), dtype=np.uint8)

    for row in prange(0, h, block_size):
        for col in range(0, w, block_size):
            # Verifica che il blocco non ecceda i limiti dell'immagine
            if row + block_size > h or col + block_size > w:
                continue

            # Estrai il blocco corrente
            current_block = current_gray[row:row + block_size, col:col + block_size]

            # Calcola il miglior match con il frame precedente
            best_sad, _, _ = three_step_search_optimized(current_block, previous_gray,
                                                         row, col, block_size, search_range)

            # Se la SAD supera la soglia, considera il blocco in movimento
            if best_sad > diff_threshold:
                mask[row:row + block_size, col:col + block_size] = 255

    return mask

def run_motion_detection(video_path):
    """
    Esegue la rilevazione del movimento su un video.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Errore: impossibile aprire il video.")
        return

    previous_gray = None
    block_size = 4
    search_range = 3
    diff_threshold = 2000

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Fine del video o errore nella lettura.")
            break

        current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if previous_gray is None:
            previous_gray = current_gray
            continue

        # Calcola la maschera di movimento
        mask = compute_motion_mask(current_gray, previous_gray,
                                   block_size=block_size,
                                   search_range=search_range,
                                   diff_threshold=diff_threshold)

        # Mostra i risultati
        cv2.imshow("Frame", frame)
        cv2.imshow("Motion Mask", mask)

        # Interrompi con 'q'
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("Rilevamento del movimento interrotto.")
            break

        # Aggiorna il frame precedente
        previous_gray = current_gray

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = '../Dataset/input/Stealing008_x264.mp4'
    run_motion_detection(video_path)


/var/folders/wk/0nbx264j3wsfdbvb01smr4t00000gn/T/ipykernel_4638/2484917105.py:59: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../../var/folders/wk/0nbx264j3wsfdbvb01smr4t00000gn/T/ipykernel_4638/2484917105.py", line 5:
<source missing, REPL/exec in use?>

  best_sad, _, _ = three_step_search_optimized(current_block, previous_gray,


UnsupportedRewriteError: Failed in nopython mode pipeline (step: convert to parfors)
Only constant step size is supported for prange

File "../../../../../../var/folders/wk/0nbx264j3wsfdbvb01smr4t00000gn/T/ipykernel_4638/2484917105.py", line 49:
<source missing, REPL/exec in use?>
